# AOP - Aspect Oriented Programming in Python

In Python, Aspect-Oriented Programming (AOP) is not as commonly used as in Java with AspectJ, but you can achive similar outcomes using decorators and libraries designed for aspect-oriented programming. Python's dynamic nature allows for flexible approaches to aspects like <b> logging, database connections, and messaging</b> without needing a dedicated AOP framework.

# 1. Logging

You can use decorators to automatically log entry and exit points of functions. This is similar to what you might do with an aspect in AspectJ.

In [1]:
import logging
from functools import wraps

# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def log_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        logger.debug(f"Entering {func.__name__} with arguments {args} and {kwargs}")
        result = func(*args, **kwargs)
        logger.debug(f"Exiting {func.__name__} with result {result}")
        return result
    return wrapper

# Usage example

@log_decorator
def example_function(x, y):
    return x + y

example_function(2, 3)

2024-08-30 10:51:21,856 - __main__ - DEBUG - Entering example_function with arguments (2, 3) and {}
2024-08-30 10:51:21,858 - __main__ - DEBUG - Exiting example_function with result 5


5

In [2]:
import logging
from functools import wraps

# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def log_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        logger.debug(f"Entering {func.__name__} with arguments {args} and keyword arguments {kwargs}")
        result = func(*args, **kwargs)
        logger.debug(f"Exiting {func.__name__} with result {result}")
        return result
    return wrapper

# Usage example with keyword arguments

@log_decorator
def example_function(x, y, operation='add'):
    if operation == 'add':
        return x + y
    elif operation == 'multiply':
        return x * y
    else:
        return None

example_function(2, 3, operation='multiply')

2024-08-30 10:52:15,386 - __main__ - DEBUG - Entering example_function with arguments (2, 3) and keyword arguments {'operation': 'multiply'}
2024-08-30 10:52:15,389 - __main__ - DEBUG - Exiting example_function with result 6


6

# 2. Database Connection

To manage database connections, you can use a decorator to handle connection setup and teardown, ensuring that resources are properly managed.

In [3]:
import sqlite3
from functools import wraps

def with_db_connection(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        connection = sqlite3.connect('example.db')
        try:
            result = func(connection, *args, **kwargs)
        finally:
            connection.close()
        return result
    return wrapper

@log_decorator
@with_db_connection
def create_table(connection):
    cursor = connection.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS my_table (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            value INTEGER
        )
    ''')
    connection.commit()

@log_decorator
@with_db_connection
def clear_table(connection):
    cursor = connection.cursor()
    cursor.execute('DELETE FROM my_table')
    connection.commit()

@log_decorator
@with_db_connection
def insert_values(connection, values):
    cursor = connection.cursor()
    cursor.executemany('''
        INSERT INTO my_table (name, value)
        VALUES (?, ?)
    ''', values)
    connection.commit()

@log_decorator
@with_db_connection
def query_database(connection, query):
    cursor = connection.cursor()
    cursor.execute(query)
    return cursor.fetchall()

# Create the table (if not exists)
create_table()

# Clear the table before inserting new values
clear_table()

# Insert some values
values_to_insert = [
    ('Alice', 30),
    ('Bob', 25),
    ('Charlie', 35)
]
insert_values(values_to_insert)

# Query the database to verify insertion
result = query_database('SELECT * FROM my_table')
print(result)


2024-08-30 10:53:01,214 - __main__ - DEBUG - Entering create_table with arguments () and keyword arguments {}
2024-08-30 10:53:01,241 - __main__ - DEBUG - Exiting create_table with result None
2024-08-30 10:53:01,243 - __main__ - DEBUG - Entering clear_table with arguments () and keyword arguments {}
2024-08-30 10:53:01,249 - __main__ - DEBUG - Exiting clear_table with result None
2024-08-30 10:53:01,249 - __main__ - DEBUG - Entering insert_values with arguments ([('Alice', 30), ('Bob', 25), ('Charlie', 35)],) and keyword arguments {}
2024-08-30 10:53:01,264 - __main__ - DEBUG - Exiting insert_values with result None
2024-08-30 10:53:01,264 - __main__ - DEBUG - Entering query_database with arguments ('SELECT * FROM my_table',) and keyword arguments {}
2024-08-30 10:53:01,281 - __main__ - DEBUG - Exiting query_database with result [(1, 'Alice', 30), (2, 'Bob', 25), (3, 'Charlie', 35)]


[(1, 'Alice', 30), (2, 'Bob', 25), (3, 'Charlie', 35)]


# 3. JMS Connection (Message Queue)

For handling JMS-like connections (e.g, message queues), you can use a similar decorator pattern to manage connections.

In [ ]:
import queue # Example of Python's build-in queue module as placeholder

def with_message_queue(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        msg_queue = queue.Queue()
        try:
            result = func(msg_queue, *args, **kwargs)
        finally:
            # clean up if necessary
            pass
        return result
    return wrapper

# Usage example

@log_decorator
@with_message_queue
def process_message(msg_queue):
    msg_queue.put('message1')
    msg_queue.put('message2')
    while not msg_queue.empty():
        message = msg_queue.get()
        print(f"Processing {message}")

process_message()

# Advanced AOP Libraries

For more advanced AOP in Python, you might explore third-party libraries:

    1. "aspectlib": A library that provides AOP-like functionality using decorators.

In [ ]:
from aspectlib import Aspect, Proceed

class LoggingAspect(Aspect):
    def around(self, proceed, *args, **kwargs):
        print(f"Entering function with args: {args}, kwargs: {kwargs}")
        result = proceed(*args, **kwargs)
        print(f"Esiting function with result:")
        return result

@LoggingAspect
def some_function(x, y):
    return x + y

some_function()
        

2. "wrapt": A library that simplifies creating decorators and managing function wrapping.

In [ ]:
import wrapt

@wrapt.decorator
def log_decorator(wrapped, instance, args, kwargs):
    print(f"Calling {wrapped.__name__} with args:{args}, kwargs: {kwargs}")
    result = wrapped(*args, **kwargs)
    print(f"{wrapped.__name__} returned {result}")
    return result

@log_decorator
def add(x, y):
    return x + y

add(5, 6)

# Summary

1. <b>Decorators:</b> Python's native way to achieve AOP-like behavior. They can be used for logging, managing database connection and more.


2. <b>Third-Party Liberaries:</b> Libraries like <b>"aspectlib"</b> and <b>"wrapt"</b> offer more advanced AOP features.


*****************************************

************************************

To implement a logging configuration for your FastAPI application using Uvicorn, you can create a dedicated logging utility and an API endpoint to configure logging levels dynamically. Below is a comprehensive setup to handle asynchronous logging for different levels (e.g., DEBUG, INFO, WARNING, ERROR, CRITICAL) and output logs to both the console and a file.

### Step 1: Set Up Logging Configuration

You can create a separate module for logging configuration, for example, `logging_config.py`.

```python
# logging_config.py
import logging
import os

def setup_logging(log_file='app.log', log_level=logging.INFO):
    # Create a logger
    logger = logging.getLogger("uvicorn")
    logger.setLevel(log_level)

    # Create handlers
    console_handler = logging.StreamHandler()
    file_handler = logging.FileHandler(log_file)

    # Set log level for handlers
    console_handler.setLevel(log_level)
    file_handler.setLevel(log_level)

    # Create formatters and add them to the handlers
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    console_handler.setFormatter(formatter)
    file_handler.setFormatter(formatter)

    # Add handlers to the logger
    logger.addHandler(console_handler)
    logger.addHandler(file_handler)
```

### Step 2: Create an API for Logging Configuration

Now, you can create an API endpoint to change the logging level dynamically. You can add this to your existing FastAPI application.

```python
# main.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import logging
from logging_config import setup_logging

app = FastAPI()

# Setup logging on startup
setup_logging()

class LogLevel(BaseModel):
    level: str

@app.post("/set-log-level/")
async def set_log_level(log_level: LogLevel):
    """API endpoint to set the log level."""
    levels = {
        "DEBUG": logging.DEBUG,
        "INFO": logging.INFO,
        "WARNING": logging.WARNING,
        "ERROR": logging.ERROR,
        "CRITICAL": logging.CRITICAL,
    }

    if log_level.level not in levels:
        raise HTTPException(status_code=400, detail="Invalid log level")

    # Update logger level
    logging.getLogger("uvicorn").setLevel(levels[log_level.level])
    return {"message": f"Log level set to {log_level.level}"}

# Example of your existing chat service function
from .chat_service import handle_query_produce_message

# Add your chat service routes and other logic here
```

### Step 3: Update Your Chat Service

You can utilize the logger within your chat service for logging various events, errors, or important information:

```python
# chat_service.py
import logging
from pydantic import BaseModel
from fastapi import HTTPException

from .chat_producer import producer_messages
from .chat_consumer import consume_messages

logger = logging.getLogger("uvicorn")

class ResponseMessage(BaseModel):
    user_id: int
    message: str

consumer_topic = 'chat_user_queries'
producer_topic = 'chat_sys_response'

async def handle_query_produce_message(user_query: str, system_response: str):
    try:
        logger.info(f"Handling query: {user_query} with system response: {system_response}")
        
        # Consume messages based on user_query
        messages = await consume_messages(consumer_topic, user_query)
        logger.debug(f"Consumed messages: {messages}")

        # Process and produce matching messages to another topic
        for msg in messages:
            response_message = ResponseMessage(user_id=int(msg["key"]), message=msg["value"])
            await producer_messages(producer_topic, int(response_message.user_id), str(system_response.message))
            logger.info(f"Produced message for user {response_message.user_id}: {system_response.message}")

    except Exception as e:
        logger.error(f"Error occurred: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))
```

### Step 4: Start the FastAPI Application with Uvicorn

You can run your FastAPI application with Uvicorn and specify the logging level if needed:

```bash
uvicorn main:app --reload --log-level info
```

### Summary

In this setup:
- **Logging Configuration**: A separate module (`logging_config.py`) is created for logging setup.
- **Dynamic Log Level API**: An endpoint (`/set-log-level/`) allows you to change the logging level dynamically.
- **Logging in Services**: The logging module is used within the chat service to log various actions and errors.

This structure allows for comprehensive logging control, making it easy to debug and monitor your FastAPI application.

************************************************************

# Implement a toggle configuration for routing logs: 

## send logs to the console during development for easy debugging and redirect them to a file in production for persistent storage and analysis.

*******************************************************

To implement a toggle configuration for logging to either console or a file in your FastAPI application using Uvicorn, you can extend the logging setup to include options for enabling or disabling console and file logging. Below is an example that demonstrates how to achieve this.

### Step 1: Update the Logging Configuration

You can modify the `setup_logging` function to accept parameters that determine whether to log to the console, to a file, or both.

```python
# logging_config.py
import logging
import os

def setup_logging(log_to_console=True, log_to_file=True, log_file='app.log', log_level=logging.INFO):
    # Create a logger
    logger = logging.getLogger("uvicorn")
    logger.setLevel(log_level)

    # Create handlers based on the configuration
    if log_to_console:
        console_handler = logging.StreamHandler()
        console_handler.setLevel(log_level)
        logger.addHandler(console_handler)

    if log_to_file:
        file_handler = logging.FileHandler(log_file)
        file_handler.setLevel(log_level)
        logger.addHandler(file_handler)

    # Create a formatter and set it for the handlers
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    for handler in logger.handlers:
        handler.setFormatter(formatter)
```

### Step 2: Create a Configuration File

You can create a configuration file (e.g., `config.py`) to manage the settings for logging.

```python
# config.py
import os

LOG_TO_CONSOLE = os.getenv("LOG_TO_CONSOLE", "true").lower() == "true"
LOG_TO_FILE = os.getenv("LOG_TO_FILE", "true").lower() == "true"
LOG_FILE = os.getenv("LOG_FILE", "app.log")
LOG_LEVEL = os.getenv("LOG_LEVEL", "INFO")
```

### Step 3: Integrate Configuration in Your Main Application

You can then use the configuration settings in your `main.py` to set up logging accordingly.

```python
# main.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import logging
from logging_config import setup_logging
from config import LOG_TO_CONSOLE, LOG_TO_FILE, LOG_FILE, LOG_LEVEL

app = FastAPI()

# Setup logging on startup based on config
setup_logging(log_to_console=LOG_TO_CONSOLE, log_to_file=LOG_TO_FILE, log_file=LOG_FILE, log_level=logging.getLevelName(LOG_LEVEL))

class LogLevel(BaseModel):
    level: str

@app.post("/set-log-level/")
async def set_log_level(log_level: LogLevel):
    """API endpoint to set the log level."""
    levels = {
        "DEBUG": logging.DEBUG,
        "INFO": logging.INFO,
        "WARNING": logging.WARNING,
        "ERROR": logging.ERROR,
        "CRITICAL": logging.CRITICAL,
    }

    if log_level.level not in levels:
        raise HTTPException(status_code=400, detail="Invalid log level")

    # Update logger level
    logging.getLogger("uvicorn").setLevel(levels[log_level.level])
    return {"message": f"Log level set to {log_level.level}"}

# Add your routes and logic here
```

### Step 4: Set Environment Variables

To control the logging behavior, you can set environment variables before running your FastAPI application. For example, you can run the following command in your terminal:

```bash
export LOG_TO_CONSOLE=true
export LOG_TO_FILE=false
export LOG_FILE='app.log'
export LOG_LEVEL='DEBUG'
```

Then, run your application using Uvicorn:

```bash
uvicorn main:app --reload
```

### Summary

In this setup:
- **Dynamic Logging Configuration**: You can enable or disable logging to the console and/or a file using environment variables.
- **Centralized Logging Setup**: The logging configuration is centralized in a separate module, making it easy to manage.
- **Log Level API**: You can dynamically change the log level through an API endpoint.

This approach allows you to have fine-grained control over your logging configuration based on different environments (development, testing, production) without changing your code.

***************************************

****************************

To create a `run_uvicorn.sh` shell script that sets the environment variables and runs your FastAPI application with Uvicorn, you can follow these steps:

### Step 1: Create the Shell Script

Create a file named `run_uvicorn.sh` in your project directory. You can use any text editor to create this file.

### Step 2: Add the Following Content to `run_uvicorn.sh`

```bash
#!/bin/bash

# Set environment variables for logging configuration
export LOG_TO_CONSOLE=true
export LOG_TO_FILE=false
export LOG_FILE='app.log'
export LOG_LEVEL='DEBUG'

# Run the FastAPI application using Uvicorn
uvicorn main:app --reload
```

### Step 3: Make the Script Executable

After creating the script, you need to make it executable. You can do this by running the following command in your terminal:

```bash
chmod +x run_uvicorn.sh
```

### Step 4: Run Your Application

Now you can run your FastAPI application by executing the script:

```bash
./run_uvicorn.sh
```

### Summary

- The `run_uvicorn.sh` script sets the necessary environment variables for logging configuration.
- It then runs the FastAPI application using Uvicorn with the specified settings.
- Making the script executable allows you to run it easily from the command line.

This approach simplifies the process of starting your application with the desired logging configuration, making it easier to manage different environments or configurations.

*******************************

************************

To set up the `run_uvicorn.sh` script in PyCharm's Run Configuration for your FastAPI application, follow these steps:

### Step 1: Create the `run_uvicorn.sh` Script

Make sure you have created the `run_uvicorn.sh` script as outlined in previous steps. It should be located in your project directory.

### Step 2: Open Run/Debug Configurations in PyCharm

1. Open PyCharm.
2. Go to the top menu and click on **Run**.
3. Select **Edit Configurations...** from the dropdown.

### Step 3: Create a New Shell Script Configuration

1. Click on the **+** icon (Add New Configuration) in the upper left corner of the Run/Debug Configurations dialog.
2. Select **Shell Script** from the list of available configurations.

### Step 4: Configure the Shell Script

1. **Name**: Give your configuration a name, such as `Run Uvicorn`.
2. **Script path**: Click the folder icon and navigate to the location of your `run_uvicorn.sh` script.
3. **Working directory**: Set the working directory to your project root or where your FastAPI application is located (where the `main.py` file is).
4. **Environment variables**: You can add any additional environment variables if needed, but since they are already set in the script, this step is optional.

### Step 5: Apply and Run

1. Click **Apply** and then **OK** to save your configuration.
2. Now, you can run your FastAPI application by selecting the newly created configuration (`Run Uvicorn`) from the dropdown menu in the top right corner of PyCharm and clicking the green run button.

### Summary

By following these steps, you've successfully set up a Run Configuration in PyCharm to execute your `run_uvicorn.sh` script. This allows you to start your FastAPI application with the desired logging configuration directly from the PyCharm IDE, simplifying the development workflow.

***********************************************

**********************************

When considering whether to use Aspect-Oriented Programming (AOP), decorators, or API logging with `asyncio` in Python, each approach has its strengths and best use cases. Below is a comparison to help you decide which method might be best for your logging or cross-cutting concerns in your FastAPI application.

### 1. Decorators

**Advantages:**
- **Simplicity**: Decorators are straightforward to implement and use. They can easily wrap functions or methods to add logging or other behavior.
- **Readability**: They enhance the readability of your code by keeping the logging logic separate from the business logic.
- **Flexibility**: You can apply them to any function or method, including class methods and standalone functions.

**Use Case**: If you want to log specific function calls with minimal overhead and clear syntax, decorators are a great choice. For instance, logging the entry and exit of specific endpoints in your FastAPI application can be done easily with a decorator.

**Example**:
```python
def log_function_call(func):
    async def wrapper(*args, **kwargs):
        print(f"Calling function: {func.__name__}")
        result = await func(*args, **kwargs)
        print(f"Function {func.__name__} finished")
        return result
    return wrapper
```

### 2. Aspect-Oriented Programming (AOP)

**Advantages:**
- **Separation of Concerns**: AOP allows you to separate logging from your business logic more cleanly. This is particularly beneficial in larger applications where cross-cutting concerns (like logging) can clutter your main code.
- **Scalability**: AOP can manage complex logging scenarios across different modules without modifying the original code.
- **Cross-Cutting Concerns**: Ideal for applications that need extensive logging, performance monitoring, or security checks across multiple modules.

**Use Case**: AOP is best suited for large applications where you have numerous logging requirements spread across various components. AOP frameworks like `AspectLib` can be used, but they might introduce complexity.

**Example**: AOP frameworks allow you to define aspects that can log before and after method executions without modifying the methods themselves.

### 3. API Logging with Asyncio

**Advantages:**
- **Asynchronous Support**: If your application is heavily asynchronous (as with FastAPI), using `asyncio`-based logging can ensure that logging operations do not block your event loop, maintaining the performance of your application.
- **Integration with Async Frameworks**: It allows you to take advantage of Python's `async` features, making it suitable for high-performance web applications.

**Use Case**: Use `asyncio` logging when your application is asynchronous and you need to maintain performance while logging requests and responses. This could be useful for logging incoming requests and their responses in a non-blocking way.

**Example**:
```python
import logging
import asyncio

async def async_log(message):
    await asyncio.sleep(0)  # Simulate async I/O operation
    logging.info(message)
```

### Conclusion

- **For Simplicity and Readability**: Use decorators if you need to log specific function calls with minimal overhead.
- **For Large Applications with Complex Logging**: Consider AOP for better separation of concerns and scalability.
- **For Asynchronous Applications**: Use `asyncio` logging to ensure that logging does not block your event loop, particularly in high-performance applications like those built with FastAPI.

Ultimately, the best choice depends on the specific requirements of your application, such as its size, complexity, and the importance of performance in your logging strategy. In many cases, decorators are often sufficient for logging in a FastAPI application, but as your application grows, you may find that AOP or async logging becomes more relevant.

******************************************************************************

*******************************************

To implement logging in a modular and packaged way, we can create a structured logging system that encapsulates the logging functionality and allows easy integration throughout your application. Below is a step-by-step implementation, including creating a logging module and using it in your async functions.

### Step 1: Create a Logging Module

Create a file named `logger.py` that will handle the configuration and setup of the logging system.

```python
# logger.py
import logging
import sys

def setup_logging(log_level=logging.INFO):
    """Setup the logging configuration."""
    logging.basicConfig(
        level=log_level,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.StreamHandler(sys.stdout),  # Log to stdout
            logging.FileHandler('app.log')      # Log to file
        ]
    )

def log_info(message):
    """Log an info message."""
    logging.info(message)

def log_error(message):
    """Log an error message."""
    logging.error(message)

# Additional logging functions can be added as needed.
```

### Step 2: Create Your Async Logging Function

Now, modify your async logging function to use the logging module we just created. Create or modify a file named `async_logger.py`.

```python
# async_logger.py
import asyncio
from logger import log_info

async def async_log(message):
    await asyncio.sleep(0)  # Simulate async I/O operation
    log_info(message)
```

### Step 3: Setting Up Your Application

Now, you can set up your main application file, which initializes logging and uses the async logging function. Create a file named `main.py`.

```python
# main.py
import asyncio
from logger import setup_logging
from async_logger import async_log

async def main():
    # Log a start message
    await async_log("Application started.")

    # Simulate some asynchronous operations
    for i in range(5):
        await async_log(f"Processing item {i + 1}.")

    # Log a completion message
    await async_log("Application finished.")

if __name__ == "__main__":
    setup_logging()  # Set up logging configuration
    asyncio.run(main())  # Run the main async function
```

### Directory Structure

Your project directory should look like this:

```
/your_project
│
├── logger.py
├── async_logger.py
└── main.py
```

### Explanation

1. **Logging Module (`logger.py`)**:
   - `setup_logging`: Configures the logging settings, including log level and format.
   - `log_info` and `log_error`: Functions to log messages at different levels.

2. **Async Logging (`async_logger.py`)**:
   - `async_log`: Asynchronously logs messages using the `log_info` function from the logging module.

3. **Main Application (`main.py`)**:
   - Initializes logging and demonstrates how to log messages in an async context.

### Usage

- Run the `main.py` file. It will log messages both to the console and to a file named `app.log`.
- You can easily extend the logging capabilities by adding more functions to the `logger.py` module as needed.

This modular approach makes it easy to maintain and scale your logging system as your application grows.

*************************************************

# asyncio logging implementation

*********************************************

To implement logging in a modular and packaged way, we can create a structured logging system that encapsulates the logging functionality and allows easy integration throughout your application. Below is a step-by-step implementation, including creating a logging module and using it in your async functions.

### Step 1: Create a Logging Module

Create a file named `logger.py` that will handle the configuration and setup of the logging system.

```python
# logger.py
import logging
import sys

def setup_logging(log_level=logging.INFO):
    """Setup the logging configuration."""
    logging.basicConfig(
        level=log_level,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.StreamHandler(sys.stdout),  # Log to stdout
            logging.FileHandler('app.log')      # Log to file
        ]
    )

def log_info(message):
    """Log an info message."""
    logging.info(message)

def log_error(message):
    """Log an error message."""
    logging.error(message)

# Additional logging functions can be added as needed.
```

### Step 2: Create Your Async Logging Function

Now, modify your async logging function to use the logging module we just created. Create or modify a file named `async_logger.py`.

```python
# async_logger.py
import asyncio
from logger import log_info

async def async_log(message):
    await asyncio.sleep(0)  # Simulate async I/O operation
    log_info(message)
```

### Step 3: Setting Up Your Application

Now, you can set up your main application file, which initializes logging and uses the async logging function. Create a file named `main.py`.

```python
# main.py
import asyncio
from logger import setup_logging
from async_logger import async_log

async def main():
    # Log a start message
    await async_log("Application started.")

    # Simulate some asynchronous operations
    for i in range(5):
        await async_log(f"Processing item {i + 1}.")

    # Log a completion message
    await async_log("Application finished.")

if __name__ == "__main__":
    setup_logging()  # Set up logging configuration
    asyncio.run(main())  # Run the main async function
```

### Directory Structure

Your project directory should look like this:

```
/your_project
│
├── logger.py
├── async_logger.py
└── main.py
```

### Explanation

1. **Logging Module (`logger.py`)**:
   - `setup_logging`: Configures the logging settings, including log level and format.
   - `log_info` and `log_error`: Functions to log messages at different levels.

2. **Async Logging (`async_logger.py`)**:
   - `async_log`: Asynchronously logs messages using the `log_info` function from the logging module.

3. **Main Application (`main.py`)**:
   - Initializes logging and demonstrates how to log messages in an async context.

### Usage

- Run the `main.py` file. It will log messages both to the console and to a file named `app.log`.
- You can easily extend the logging capabilities by adding more functions to the `logger.py` module as needed.

This modular approach makes it easy to maintain and scale your logging system as your application grows.

**************************************************

******************************

To implement sequential logging using `asyncio`, you can use a `deque` (double-ended queue) from the `collections` module instead of a `set`. A `deque` allows you to efficiently append and pop items from both ends, making it suitable for managing a queue of log messages that need to be processed in order.

Here’s how you can implement sequential logging with `asyncio` using a `deque`:

### Step 1: Create Your Logging Queue Module

Create a file named `logging_queue.py` that will manage a queue for log messages.

```python
# logging_queue.py
import asyncio
from collections import deque

class LoggingQueue:
    def __init__(self):
        self.queue = deque()  # Initialize a deque to hold log messages
        self.lock = asyncio.Lock()  # Lock for thread-safe access

    async def add_log(self, message):
        async with self.lock:
            self.queue.append(message)

    async def process_logs(self):
        while True:
            async with self.lock:
                if self.queue:
                    message = self.queue.popleft()  # Get the next log message
                    print(message)  # Replace this with your logging mechanism
                else:
                    await asyncio.sleep(1)  # Wait before checking again
```

### Step 2: Create Your Async Operations Module

Next, modify your `async_operations.py` file to use the `LoggingQueue`.

```python
# async_operations.py
import asyncio
from logging_queue import LoggingQueue

logging_queue = LoggingQueue()

async def process_items(num_items):
    """Process a number of items asynchronously."""
    for i in range(num_items):
        message = f"Processing item {i + 1}."
        await logging_queue.add_log(message)  # Add log message to the queue
        await asyncio.sleep(1)  # Simulate some processing time
```

### Step 3: Modify Your Main Application File

Now, update your `main.py` file to include the log processing.

```python
# main.py
import asyncio
from logger import setup_logging
from async_operations import process_items
from logging_queue import logging_queue

async def main():
    # Start the log processing in the background
    log_processor = asyncio.create_task(logging_queue.process_logs())

    # Log a start message
    await logging_queue.add_log("Application started.")

    # Process items asynchronously
    await process_items(5)

    # Log a completion message
    await logging_queue.add_log("Application finished.")

    # Wait for a moment to ensure all logs are processed before exiting
    await asyncio.sleep(2)

    # Cancel the log processor task
    log_processor.cancel()

if __name__ == "__main__":
    setup_logging()  # Set up logging configuration
    asyncio.run(main())  # Run the main async function
```

### Directory Structure

Your project directory should look like this:

```
/your_project
│
├── logger.py
├── async_logger.py
├── async_operations.py
├── logging_queue.py
└── main.py
```

### Explanation

1. **Logging Queue Module (`logging_queue.py`)**:
   - `LoggingQueue` class manages a deque to store log messages.
   - `add_log`: Asynchronously adds a log message to the queue.
   - `process_logs`: Continuously processes and prints log messages from the queue.

2. **Async Operations Module (`async_operations.py`)**:
   - Uses the `LoggingQueue` to add log messages asynchronously.

3. **Main Application File (`main.py`)**:
   - Starts the log processing task in the background.
   - Adds log messages at the start and end of the application.
   - Ensures all log messages are processed before the application exits.

### Advantages of Using `deque`

- **Efficient**: `deque` allows O(1) time complexity for appending and popping elements from either end.
- **Ordered**: Unlike a `set`, a `deque` maintains the order of messages, ensuring they are processed sequentially.

### Usage

- Run the `main.py` file. It will log messages sequentially as items are processed.
- You can further expand the functionality of the logging system by modifying the `LoggingQueue` class or adding additional logging features.